- Bayesian linear regression for a single step ahead (Sunspot time series) and multi-step ahead time series prediction (MMM stock market). Below you can see trace-plot and histogram of the posterior distribution. Note that Bayesian multinomial regression can be developed using this code for multi-class classification problems. 

- We note that in multi-step time series prediction, 5 step-ahead would mean 5 output neurons. We use sigmoid units in the output layer. Note that gradients are not used and you can compare the results with SGD which is present in the code.

In [ ]:
# by R. Chandra
# https://github.com/rohitash-chandra/Bayesianlogisticreg_multioutputs

import random
import numpy as np
import math
from math import exp
import matplotlib.pyplot as plt


class lin_model:

    def __init__(self, num_epocs, train_data, test_data, num_features, learn_rate, activation):
        self.train_data = train_data
        self.test_data = test_data 
        self.num_features = num_features
        self.num_outputs = self.train_data.shape[1] - num_features 
        self.num_train = self.train_data.shape[0]

        #self.w = np.random.uniform(-0.5, 0.5, num_features)  # in case one output class
        self.w = np.random.uniform(-.5, .5, (num_features, self.num_outputs))  
        self.b = np.random.uniform(-.5, .5, self.num_outputs) 
        self.learn_rate = learn_rate
        self.max_epoch = num_epocs
        self.use_sigmoid = activation    # SIGMOID: 1 -sigmoid, 2 -step, 3 -linear 
        self.out_delta = np.zeros(self.num_outputs)

        print(self.w, ' self.w init') 
        print(self.b, ' self.b init')        

    def activation_func(self,z_vec):
        if self.use_sigmoid == True:
            y = 1 / (1 + np.exp(z_vec))   # sigmoid/logistic 
        else:  
            y = z_vec
        return y
    

    def squared_error(self, prediction, actual):
        return  np.sum(np.square(prediction - actual))/prediction.shape[0]   # to cater more in one output/class
    

    def predict(self, x_vec ):    # implementation using dot product
        z_vec = x_vec.dot(self.w) - self.b 
        output = self.activation_func(z_vec)    # Output 
        return output
    

    def gradient(self, x_vec, output, actual):   
        if self.use_sigmoid == True :
            out_delta = (output - actual)  *(output * (1 - output)) 
        else:    # for linear activation
            out_delta = output - actual
        return out_delta  
    

    def update(self, x_vec, output, actual):    # implementation using for loops 
        for x in range(0, self.num_features):
            for y in range(0, self.num_outputs):
                self.w[x,y] += self.learn_rate * self.out_delta[y] * x_vec[x] 
        for y in range(0, self.num_outputs):
            self.b += -1 * self.learn_rate * self.out_delta[y]
            

    def test_model(self, data, tolerance):  

        num_instances = data.shape[0]

        class_perf = 0
        sum_sqer = 0   
        for s in range(0, num_instances):
            input_instance = self.train_data[s,0:self.num_features] 
            actual = self.train_data[s,self.num_features:]  
            prediction = self.predict(input_instance) 
            pred_binary = np.zeros(prediction.shape[0])
            sum_sqer += self.squared_error(prediction, actual)
            index = np.argmax(prediction)
            pred_binary[index] = 1    # i= for softmax  
            #pred_binary = np.where(prediction > (1 - tolerance), 1, 0) # for sigmoid in case of classification

            if (actual==pred_binary).all():
                class_perf += 1   

        rmse = np.sqrt(sum_sqer/num_instances)
        percentage_correct = float(class_perf/num_instances) * 100 
        print(percentage_correct, rmse,  ' class_perf, rmse')  

        return (rmse, percentage_correct)


    def SGD(self):   
        epoch = 0 
        shuffle = True

        while epoch < self.max_epoch:
            sum_sqer = 0
            for s in range(0, self.num_train):
                if shuffle==True:
                    i = random.randint(0, self.num_train-1) 
                    input_instance = self.train_data[i, 0:self.num_features]  
                    actual = self.train_data[i, self.num_features:]  
                    prediction = self.predict(input_instance) 
                    sum_sqer += self.squared_error(prediction, actual)
                    self.out_delta = self.gradient(input_instance, prediction, actual)
                    # major difference when compared to GD 
                    self.update(input_instance, prediction, actual) 

            epoch += 1  

        rmse_train, train_perc = self.test_model(self.train_data, 0.3) 
        rmse_test = 0
        test_perc = 0
        rmse_test, test_perc = self.test_model(self.test_data, 0.3)

        return (train_perc, test_perc, rmse_train, rmse_test) 

    # new functions added for MCMC below

    def encode(self, w):    # get the parameters and encode into the model
        w_size = self.num_features * self.num_outputs
        w_temp = w[0:w_size]
        self.w = np.reshape(w_temp, (self.num_features, self.num_outputs))
        self.b = w[w_size:w.shape[0]]
        

    def evaluate_proposal(self, data, w):   # BP with SGD (Stocastic BP)

        self.encode(w)    # method to encode w and b
        fx = np.zeros((data.shape[0], self.num_outputs))

        for s in range(0, data.shape[0]):
            i = s
            #random.randint(0, data.shape[0]-1)  (we dont shuffle in this implementation)
            input_instance = data[i,0:self.num_features]  
            actual = data[i,self.num_features:]  
            prediction = self.predict(input_instance)  
            fx[s, :] = prediction 

        return fx

#------------------------------------------------------------------

class MCMC:
    def __init__(self, samples, traindata, testdata, topology, regression):
        self.samples = samples    # NN topology [input, hidden, output]
        self.topology = topology  # max epocs
        self.traindata = traindata
        self.testdata = testdata
        random.seed() 
        self.regression = regression # False means classification


    def rmse(self, predictions, targets):
        return np.sqrt(((predictions - targets) ** 2).mean())
    

    def likelihood_func(self, model, data, w, tausq):
        y = data[:, self.topology[0]:]
        fx = model.evaluate_proposal(data, w) 
        accuracy = self.rmse(fx, y)     # RMSE 
        loss = np.sum(-0.5 * np.log(2 * math.pi * tausq) - 0.5 * np.square(y-fx)/tausq)
        return [loss, fx, accuracy]
    

    def prior_likelihood(self, sigma_squared, nu_1, nu_2, w, tausq): 
        param = (self.topology[0]  * self.topology[1]) + self.topology[1]   # number of parameters in model
        part1 = -1 * (param / 2) * np.log(sigma_squared)
        part2 = 1 / (2 * sigma_squared) * (sum(np.square(w)))
        log_loss = part1 - part2 - (1 + nu_1) * np.log(tausq) - (nu_2 / tausq)
        return log_loss
    

    def sampler(self):
        # Initialize MCMC
        testsize = self.testdata.shape[0]
        trainsize = self.traindata.shape[0]
        samples = self.samples

        x_test = np.linspace(0, 1, num=testsize)
        x_train = np.linspace(0, 1, num=trainsize)

        #self.topology  # [input, output]
        y_test = self.testdata[:, self.topology[0]:]
        y_train = self.traindata[:, self.topology[0]:]

        w_size = (self.topology[0] * self.topology[1]) + self.topology[1]  # num of weights and bias  

        pos_w = np.ones((samples, w_size))  # posterior of all weights and bias over all samples
        pos_tau = np.ones((samples, 1))

        fxtrain_samples = np.ones((samples, trainsize, self.topology[1]))  # fx of train data over all samples
        fxtest_samples = np.ones((samples, testsize, self.topology[1]))    # fx of test data over all samples
        rmse_train = np.zeros(samples)
        rmse_test = np.zeros(samples)

        w = np.random.randn(w_size)

        w_proposal = np.random.randn(w_size)

        step_w = 0.02   # defines how much variation you need in changes to w
        step_eta = 0.01  
        # eta is an additional parameter to cater for noise in predictions (Gaussian likelihood). 
        # note eta is used as tau in the sampler to consider log scale. 
        # eta is not used in multinomial likelihood. 

        model = lin_model(0, self.traindata, self.testdata, self.topology[0], 0.1, self.regression) 

        pred_train = model.evaluate_proposal(self.traindata, w) 
        pred_test = model.evaluate_proposal(self.testdata, w)

        eta = np.log(np.var(pred_train - y_train))    # this is to estimate var of eta that represents noise
        tau_pro = np.exp(eta)

        print(tau_pro, ' tau_pro')

        sigma_squared = 5
        # considered by looking at distribution of similar trained models - i.e. distribution of weights and bias
        nu_1 = 0
        nu_2 = 0

        prior_likelihood = self.prior_likelihood(sigma_squared, nu_1, nu_2, w, tau_pro)  # takes care of the gradients

        [likelihood, pred_train, rmsetrain] = self.likelihood_func(model, self.traindata, w, tau_pro)
        print(likelihood, ' initial likelihood')
        
        [likelihood_ignore, pred_test, rmsetest] = self.likelihood_func(model, self.testdata, w, tau_pro)

        naccept = 0  

        for i in range(samples - 1):
            
            w_proposal = w + np.random.normal(0, step_w, w_size)

            eta_pro = eta + np.random.normal(0, step_eta, 1)
            tau_pro = math.exp(eta_pro)

            [likelihood_proposal, pred_train, rmsetrain] = self.likelihood_func(model, self.traindata, w_proposal, tau_pro)
            [likelihood_ignore, pred_test, rmsetest] = self.likelihood_func(model, self.testdata, w_proposal, tau_pro)

            # likelihood_ignore  refers to parameter that will not be used in the alg.

            prior_prop = self.prior_likelihood(sigma_squared, nu_1, nu_2, w_proposal, tau_pro)
            # takes care of the gradients

            diff_likelihood = likelihood_proposal - likelihood
            # since we using log scale: based on https://www.rapidtables.com/math/algebra/Logarithm.html
            diff_priorliklihood = prior_prop - prior_likelihood

            mh_prob = min(1, math.exp(diff_likelihood + diff_priorliklihood))

            u = random.uniform(0, 1)

            if u < mh_prob:
                # Update position
                #print(i, ' is accepted sample')
                naccept += 1
                likelihood = likelihood_proposal
                prior_likelihood = prior_prop
                w = w_proposal
                eta = eta_pro
                rmse_train[i + 1,] = rmsetrain
                rmse_test[i + 1,] = rmsetest

                print (i, likelihood, prior_likelihood, rmsetrain, rmsetest, ' accepted')

                pos_w[i + 1,] = w_proposal
                pos_tau[i + 1,] = tau_pro
                fxtrain_samples[i + 1,] = pred_train
                fxtest_samples[i + 1,] = pred_test 

            else:
                pos_w[i + 1,] = pos_w[i,]
                pos_tau[i + 1,] = pos_tau[i,]
                fxtrain_samples[i + 1,] = fxtrain_samples[i,]
                fxtest_samples[i + 1,] = fxtest_samples[i,] 
                rmse_train[i + 1,] = rmse_train[i,]
                rmse_test[i + 1,] = rmse_test[i,]

        accept_ratio = naccept / (samples * 1.0) * 100

        print(accept_ratio, '% was accepted')

        burnin = 0.25 * samples  # use post burn in samples

        pos_w = pos_w[int(burnin):, ]
        pos_tau = pos_tau[int(burnin):, ] 
        rmse_train = rmse_train[int(burnin):]
        rmse_test = rmse_test[int(burnin):]

        rmse_tr = np.mean(rmse_train)
        rmsetr_std = np.std(rmse_train)
        rmse_tes = np.mean(rmse_test)
        rmsetest_std = np.std(rmse_test)
        print(rmse_tr, rmsetr_std, rmse_tes, rmsetest_std, ' rmse_tr, rmsetr_std, rmse_tes, rmsetest_std')

        # let us next test the Bayesian model using the posterior distributions over n trials

        num_trials = 10

        accuracy = np.zeros(num_trials)

        for i in range(num_trials):
            #print(pos_w.mean(axis=0), pos_w.std(axis=0), ' pos w mean, pos w std')
            w_drawn = np.random.normal(pos_w.mean(axis=0), pos_w.std(axis=0), w_size)
            tausq_drawn = np.random.normal(pos_tau.mean(), pos_tau.std())
            # a buf is present here - gives negative values at times

            [loss, fx_, accuracy[i]] = self.likelihood_func(model, self.testdata, w_drawn, tausq_drawn)

            print(i, loss, accuracy[i], tausq_drawn, pos_tau.mean(), pos_tau.std(), ' posterior test ')

        print(accuracy.mean(), accuracy.std(), ' is mean and std of accuracy rmse test')

        return (pos_w, pos_tau, fxtrain_samples, fxtest_samples, rmse_train, rmse_test, accept_ratio)
    
    
def histogram_trace(pos_points, fname):    # this is function to plot (not part of class)
    
    size = 15

    plt.tick_params(labelsize=size)
    params = {'legend.fontsize': size, 'legend.handlelength': 2}
    plt.rcParams.update(params)
    plt.grid(alpha=0.75)

    plt.hist(pos_points, bins = 20, color='#0504aa', alpha=0.7)   
    plt.title("Posterior distribution ", fontsize = size)
    plt.xlabel('Parameter value', fontsize = size)
    plt.ylabel('Frequency', fontsize = size) 
    plt.tight_layout()  
    plt.savefig(fname + '_posterior.png')
    plt.show()
    plt.clf()

    plt.tick_params(labelsize=size)
    params = {'legend.fontsize': size, 'legend.handlelength': 2}
    plt.rcParams.update(params)
    plt.grid(alpha=0.75) 
    plt.plot(pos_points)
    plt.title("Parameter trace plot", fontsize = size)
    plt.xlabel('Number of Samples', fontsize = size)
    plt.ylabel('Parameter value', fontsize = size)
    plt.tight_layout()  
    plt.savefig(fname + '_trace.png')
    plt.show()
    plt.clf()

    
def main():
    
    outres = open('datasets/results2.txt', 'w')
    
    for problem in range(2, 3):
        
        if problem == 1:    # Single step ahead prediction
            traindata = np.loadtxt("datasets/Sunspot/train.txt")
            testdata = np.loadtxt("datasets/Sunspot/test.txt")
            features = 4
            output = 1
            activation = False     # true for sigmoid, false for linear

        if problem == 2:
            # Multi-step ahead prediction
            # MMM stock market - 5 steps ahead predicton for closing stock price
            # https://au.finance.yahoo.com/quote/mmm/
            traindata = np.loadtxt("datasets/Stockmarket/train.txt")
            testdata = np.loadtxt("datasets/Stockmarket/test.txt")
            features = 5
            output = 5
            activation = False   # true for sigmoid, false for linear

        print(traindata)

        topology = [features, output]

        model = lin_model(500, traindata, testdata, topology[0], 0.1, activation) 

        train_perc, test_perc, rmse_train, rmse_test = model.SGD()

        print(train_perc, test_perc, rmse_train, rmse_test, ' train_perc, test_perc, rmse_train, rmse_test using SGD')

        #--------------------------------------------------

        MinCriteria = 0.005  # stop when RMSE reaches MinCriteria (problem dependent)

        numSamples = 5000    # need to decide yourself

        mcmc = MCMC(numSamples, traindata, testdata, topology, activation)  # declare class

        [pos_w, pos_tau, fx_train, fx_test, rmse_train, rmse_test, accept_ratio] = mcmc.sampler()

        fx_mu = fx_test.mean(axis=0)
        fx_high = np.percentile(fx_test, 95, axis=0)
        fx_low = np.percentile(fx_test, 5, axis=0)
        fx_mu_tr = fx_train.mean(axis=0)
        fx_high_tr = np.percentile(fx_train, 95, axis=0)
        fx_low_tr = np.percentile(fx_train, 5, axis=0)

        rmse_tr = np.mean(rmse_train)
        rmsetr_std = np.std(rmse_train)
        rmse_tes = np.mean(rmse_test)
        rmsetest_std = np.std(rmse_test)

        np.savetxt(outres, (rmse_tr, rmsetr_std, rmse_tes, rmsetest_std, accept_ratio), fmt='%1.5f')

        ytestdata = testdata[:, features]
        ytraindata = traindata[:, features]
        x_test = np.linspace(0, 1, num=testdata.shape[0])
        x_train = np.linspace(0, 1, num=traindata.shape[0])

        '''plt.plot(x_test, ytestdata, label='actual')
        plt.plot(x_test, fx_mu, label='pred.(mean)')
        plt.plot(x_test, fx_low, label='pred.(5th percen.)')
        plt.plot(x_test, fx_high, label='pred.(95th percen.)')
        plt.fill_between(x_test, fx_low, fx_high, facecolor='g', alpha=0.4)
        plt.legend(loc='upper right')
        plt.title("Test Data Uncertainty ")
        plt.savefig('figures/mcmctest.png')
        plt.show()
        plt.clf()
        
        # -----------------------------------------
        
        plt.plot(x_train, ytraindata, label='actual')
        plt.plot(x_train, fx_mu_tr, label='pred.(mean)')
        plt.plot(x_train, fx_low_tr, label='pred.(5th percen.)')
        plt.plot(x_train, fx_high_tr, label='pred.(95th percen.)')
        plt.fill_between(x_train, fx_low_tr, fx_high_tr, facecolor='g', alpha=0.4)
        plt.legend(loc='upper right')
        plt.title("Train Data Uncertainty ")
        plt.savefig('figures/mcmctrain.png')
        plt.show()
        plt.clf()'''

        mpl_fig = plt.figure()
        ax = mpl_fig.add_subplot(111)
        ax.boxplot(pos_w)
        ax.set_xlabel('[w0] [w1] [w3] [b]')
        ax.set_ylabel('Posterior')
        plt.legend(loc='upper right')
        plt.title("Posterior")
        plt.savefig('figures/w_pos.png')
        plt.show()
        plt.clf()

        import os
        folder = 'figures'
        if not os.path.exists(folder):
            os.makedirs(folder)

        #for i in range(pos_w.shape[1]):

        for i in range(5):
            histogram_trace(pos_w[:,i], folder+'/'+str(i))
            

if __name__ == "__main__":
    main() 

[[5.54000e-04 3.73900e-03 1.98500e-03 ... 4.20000e-03 1.06200e-03
  3.97000e-03]
 [1.98500e-03 0.00000e+00 2.30800e-03 ... 3.97000e-03 7.84700e-03
  1.12160e-02]
 [2.30800e-03 4.29300e-03 1.84600e-03 ... 1.12160e-02 1.05240e-02
  1.03390e-02]
 ...
 [6.93515e-01 7.01704e-01 6.94112e-01 ... 6.94491e-01 6.76650e-01
  6.92593e-01]
 [6.94112e-01 7.09296e-01 6.94166e-01 ... 6.92593e-01 6.84730e-01
  6.97528e-01]
 [6.94166e-01 6.92539e-01 6.96552e-01 ... 6.97528e-01 7.05500e-01
  7.06259e-01]]
[[ 0.19501245 -0.42787433 -0.44500019 -0.42245869  0.08695229]
 [-0.09754217  0.12578379 -0.29525026  0.30677844  0.00785604]
 [ 0.17812435  0.15360126  0.35716324 -0.45325319 -0.3546039 ]
 [ 0.24864554  0.37547593  0.13433643 -0.49453747 -0.13222017]
 [-0.1182563  -0.04220437 -0.23727212  0.14013973  0.26251846]]  self.w init
[ 0.43389103 -0.4185525  -0.37356592  0.37067382  0.27667912]  self.b init


C:\Users\di_ma\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\di_ma\AppData\Local\Temp/ipykernel_11208/1386866908.py:40: RuntimeWarning: overflow encountered in square
  return  np.sum(np.square(prediction - actual))/prediction.shape[0]   # to cater more in one output/class
C:\Users\di_ma\AppData\Local\Temp/ipykernel_11208/1386866908.py:62: RuntimeWarning: overflow encountered in add
  self.b += -1 * self.learn_rate * self.out_delta[y]


0.0 nan  class_perf, rmse
0.0 nan  class_perf, rmse
0.0 0.0 nan nan  train_perc, test_perc, rmse_train, rmse_test using SGD
[[ 0.36494371  0.36991867 -0.05431822 -0.11429268 -0.45060512]
 [-0.28451835 -0.45220988 -0.16419805  0.16286393  0.41036738]
 [-0.1481562   0.07515119 -0.21526428  0.12937607 -0.00149895]
 [ 0.43686832 -0.49307152 -0.38069026  0.43669857 -0.34674241]
 [ 0.16502611  0.02880121 -0.47588164 -0.27124769  0.1523828 ]]  self.w init
[-0.42062668  0.0803283  -0.02793981  0.11972065  0.30663819]  self.b init
0.12950681444079562  tau_pro
-3242.734288225582  initial likelihood
0 -3072.1276649638476 -24.452419437721623 0.4736727126357085 0.4740308078279326  accepted
1 -3037.944415359088 -24.45641673134471 0.47085775527130047 0.471353658631431  accepted
4 -2935.1105244659816 -24.443571168926482 0.4640402258635376 0.46488326531968516  accepted
5 -2884.7337295813973 -24.407031552608256 0.4606842637141537 0.4636974540567485  accepted
6 -2648.410621407719 -24.384593757862312 0.44

225 703.2304189473792 -24.029301796789746 0.1236247207600117 0.11451147100408145  accepted
227 738.1304116458753 -23.970605869020556 0.12157009782089798 0.12554769415200662  accepted
232 748.648463489893 -23.967313064192115 0.11903433913603018 0.10932014695538975  accepted
240 795.2643792482075 -23.939407625956697 0.11489317876052581 0.1169390821332097  accepted
242 803.570857855781 -23.951014808387825 0.11400848652678452 0.12068736576153585  accepted
251 822.3673579245306 -23.9419835422665 0.11427859941877232 0.12266064366048418  accepted
255 843.6418512775073 -23.933982300381512 0.10923780250476448 0.1182789573545593  accepted
257 856.2406682546011 -23.937780181871627 0.10551906631021177 0.10668891315269087  accepted
258 901.9898941210105 -23.934692945464665 0.0958477342060205 0.09883116397022153  accepted
259 939.7050744254129 -23.90898093889886 0.09496385136493467 0.08661669988686257  accepted
261 962.459184591617 -23.908069086682076 0.09697926928845933 0.07838034475043366  accepte

584 2479.9563323867715 -23.594684271631614 0.04390528031733105 0.05168763276178361  accepted
586 2515.557325414311 -23.559449156660197 0.04212776773235903 0.05297352028351965  accepted
604 2519.1060012467833 -23.568001181495227 0.042257212614804814 0.053739994483334025  accepted
608 2555.320704179697 -23.53577978831263 0.040005052793810954 0.05187940651356146  accepted
616 2565.8001419351417 -23.53624494249825 0.035374608997595036 0.04925512630611083  accepted
629 2575.727398838475 -23.566972109820114 0.03610754162204258 0.050697095367515974  accepted
640 2593.5155033497385 -23.55979928813301 0.0372973515314413 0.05496435426536699  accepted
651 2607.348046113092 -23.602662508117167 0.03292958989669953 0.0493703648594076  accepted
663 2619.9151968480955 -23.589843326517705 0.03922034588566611 0.06175309325032651  accepted
691 2641.386799230987 -23.56274983072723 0.040591788358361865 0.06169001944084971  accepted
692 2643.3966814403398 -23.54067411271912 0.04408789418116465 0.06160814122

1305 3854.99769321192 -22.984263041305688 0.038619060868909934 0.05637644445523082  accepted
1310 3859.4655301921557 -23.006862880530967 0.03838450422116161 0.05263498008854491  accepted
1339 3875.130228841692 -23.016912789684948 0.03871048576184786 0.055481009582146905  accepted
1373 3878.1069012719877 -22.990753151677424 0.03967873658703745 0.05700482288795451  accepted
1384 3892.429515952526 -22.996424535968288 0.03739499442132231 0.052478552784774105  accepted
1396 3918.133375479514 -22.97945527303722 0.0352451448638767 0.05522892027878755  accepted
1399 3923.9131602937796 -22.967449906937777 0.037322078050085104 0.057319890007435745  accepted
1403 3950.060459645799 -22.96519204364226 0.03447362829530647 0.055290768034863304  accepted
1427 3954.4587759494116 -22.97557740486744 0.03810316877156344 0.05400550840238203  accepted
1434 3980.808701470977 -22.952648884129403 0.03758414932624473 0.059463008607172914  accepted
1450 3994.201920880667 -22.91642786944951 0.03691941882305105 0.

2826 5176.917887803002 -22.384823638698293 0.027919387469500706 0.04596568297776676  accepted
2878 5192.6452561225915 -22.38909516589142 0.027492980887863238 0.04847999985533537  accepted
3006 5205.943516849878 -22.34016742620654 0.028577135158069287 0.0469953401851691  accepted
3030 5211.675992540131 -22.304776839004603 0.02735215694101639 0.04507340087045947  accepted
3047 5216.196895517028 -22.293142462135677 0.02920277357298439 0.04698083909867471  accepted
3049 5248.2499594209585 -22.26557561957023 0.029750284757534225 0.046031758010794635  accepted
3050 5269.702184270237 -22.211834725924366 0.029447701262947566 0.04768124206139051  accepted
3053 5331.007848105908 -22.243329649263472 0.026119441259764547 0.043975882610545254  accepted
3230 5329.170496260678 -22.2345337119706 0.026998211522503642 0.046777728644893934  accepted
3256 5343.184990160526 -22.206836482921133 0.02701580333659274 0.046428394684715726  accepted
3263 5355.255602091729 -22.179955891251094 0.02732521879291697 

No handles with labels found to put in legend.
